In [6]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [7]:
# importing libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.image import imread
import cv2

In [8]:
img_w = 128       # image width
img_h = 128       # image height
img_size = (img_w, img_h)
img_c = 3         # image color channel - RGB

In [9]:
import zipfile
zip_ref =  zipfile.ZipFile("../input/dogs-vs-cats/train.zip", 'r')
zip_ref.extractall("")

In [13]:
os.listdir("./train")[1].split('.')[0]

In [14]:
filenames = os.listdir("./train")
categories = []
for f_name in filenames:
    category = f_name.split('.')[0]
    if category == 'dog':
        categories.append(1)
    else:
        categories.append(0)
df=pd.DataFrame({
    'filename':filenames,
    'category':categories
})

In [17]:
df.head(10)

In [19]:
zip_ref =  zipfile.ZipFile("../input/dogs-vs-cats/test1.zip", 'r')
zip_ref.extractall("")

In [21]:
os.listdir('./test1')[:5]

In [25]:
test_filenames = os.listdir("./test1")
test_df = pd.DataFrame({'filename': test_filenames})
test_df[:5]

In [26]:
print("The number of images in training data: ", df.shape[0])
print("The number of images in test data: ", test_df.shape[0])

In [27]:
from sklearn.model_selection import train_test_split

df["category"] = df["category"].replace({0:'cat',1:'dog'})
train_df,validate_df = train_test_split(df,test_size=0.20, random_state=42)

train_df = train_df.reset_index(drop=True)
validate_df = validate_df.reset_index(drop=True)

print("The shape of the training data: ", train_df.shape[0])
print("The shape of the validation data: ", validate_df.shape[0])

In [28]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_img_gen = ImageDataGenerator(rotation_range=16,
                                  rescale=1/255,
                                  shear_range=0.1,
                                  zoom_range=0.1,
                                  horizontal_flip=True,
                                  width_shift_range=0.1,
                                  height_shift_range=0.1)

validation_img_gen = ImageDataGenerator(rescale=1/255)

In [29]:
train_df[:5]

In [30]:
train_gen = train_img_gen.flow_from_dataframe(train_df, 
                                              directory="./train/", 
                                              x_col='filename', 
                                              y_col='category',
                                              target_size=img_size,
                                              color_mode='rgb',
                                              class_mode='categorical',
                                              batch_size=16)

val_gen = validation_img_gen.flow_from_dataframe(validate_df,
                                                 directory="./train/", 
                                                 x_col='filename',
                                                 y_col='category',
                                                target_size=img_size,
                                                color_mode='rgb',
                                                class_mode='categorical',
                                                batch_size=16)

In [31]:
test_datagen = ImageDataGenerator(rotation_range=16,
                                rescale=1./255,
                                shear_range=0.1,
                                zoom_range=0.2,
                                horizontal_flip=True,
                                width_shift_range=0.1,
                                height_shift_range=0.1)

In [33]:
test_generator = test_datagen.flow_from_dataframe(test_df,
                                                   "./test1/",
                                                   x_col="filename",
                                                   y_col=None,
                                                   target_size=img_size,
                                                   class_mode=None,
                                                   batch_size=16)

In [35]:
# building the model

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense, BatchNormalization, Dropout

model = Sequential()
model.add(Conv2D(filters=32, kernel_size=(3,3), input_shape = (img_w, img_h, img_c), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(filters=64, kernel_size=(3,3), input_shape = (img_w, img_h, img_c), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(filters=128, kernel_size=(3,3), input_shape = (img_w, img_h, img_c), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())

model.add(Dense(units=512, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(rate=0.5))

model.add(Dense(units=2, activation='softmax'))

In [36]:
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [37]:
model.summary()

In [38]:
from tensorflow.keras.callbacks import EarlyStopping
early_stop = EarlyStopping(monitor='val_loss', patience=2)

In [39]:
result = model.fit(train_gen, epochs=10, validation_data=val_gen, callbacks=[early_stop])

In [43]:
# evaluation of the model

pred = model.predict(test_generator)

In [50]:
test_df['category'] = np.argmax(pred, axis=-1)

label_map = dict((v,k) for k,v in train_gen.class_indices.items())
test_df['category'] = test_df['category'].replace(label_map)

test_df['category'] = test_df['category'].replace({ 'dog': 1, 'cat': 0 })

In [53]:
from keras.preprocessing.image import load_img
sample_test = test_df.head(18)
sample_test.head()
plt.figure(figsize=(12, 24))
for index, row in sample_test.iterrows():
    filename = row['filename']
    category = row['category']
    img = load_img("./test1/"+filename, target_size=img_size)
    plt.subplot(6, 3, index+1)
    plt.imshow(img)
    plt.xlabel(filename + '(' + "{}".format(category) + ')' )
plt.tight_layout()
plt.show()

In [54]:
results={
    0:'cat',
    1:'dog'
}
from PIL import Image
import numpy as np
im=Image.open("./test1/4134.jpg")
im=im.resize(img_size)
im=np.expand_dims(im,axis=0)
im=np.array(im)
im=im/255
pred=model.predict([im])[0]
classes_x=np.argmax(pred,axis=0)
print(classes_x,results[classes_x])